In [1]:
#network X implementation

#things to do:
#bipartite projections
#network of islands with (1/?) distance as edge weights -> most central, etc...
#network of same species, but one node for each island 

#using https://gist.github.com/lsheble/fb0c48d2462e3cc3c8e0
#https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.bipartite.html
#could redundancy be useful? 

#1.1 can do bipartite structure, but nodes are cramped
#try https://stackoverflow.com/questions/3567018/how-can-i-specify-an-exact-output-size-for-my-networkx-graph

#1.2
#plots look good, trying to construct DFs

#1.3
#using pymnet
#need to be careful with bipartite network parameters (don't know if they consider bipartite...)

#1.4
#trying multinetx also. had to change multinetx-master folder to just multinetx
#looks like theres a problem when the number of nodes is not the same across layers

#so, include all nodes in every graph, but then pass only the relevant ones in nodelist argument? 

#1.5 working on putting every node in every graph, will need to retain the original nodelists though

#multinetx 1.0
#trying a multinetx implementation 

#need to fill out adjacency matrices with all possible pollinators and plants
#do rows first, then add 0 columns to each 
#need to retain original lists? 

#need to sort all DFs in the same way so nodes are labeled consistently when doing multiplex

#1.1
#procedure for appending rows seems good, now put into loop
#using "class" column to record which kind of node. but dictionaries are random? so maybe not right 
#that approach didn't work 
#had to correct draw.pyc from circylar network to circular with text editor 

#1.2
#looks like the layout is returning a dictionary
#changing pos -> pos.values() to allow the pos[][] to work with diff kinds of layouts (circ, fred thing)\


#1.4 source code seems ok, now try to plot 
#plot looks good
#now, need to hide nodes and (interlayer) edges that don't exist
#should also do DF calculations for un-padded data

#for bipartite layout, looks like the update is adding new "named nodes" instead of
#changing the numerical ID node locations

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%reload_ext autoreload
%autoreload 2
import scipy.stats as st
import os
from scipy.optimize import curve_fit
import igraph as ig
import csv
import networkx as nx
from networkx.algorithms import bipartite


In [3]:
adj = []
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site1_WesternSahara1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site2_WesternSahara2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site3_Fuerteventura1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site4_Fuerteventura2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site5_GranCanaria1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site6_GranCanaria2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site7_TenerifeSouth1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site8_TenerifeSouth2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site9_TenerifeTeno1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site10_TenerifeTeno2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site11_Gomera1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site12_Gomera2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site13_Hierro1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site14_Hierro2.csv"))

In [4]:
#base graphs before padding
adj_base = list(adj)

In [5]:
cols = []
for i in range(len(adj)):
    cols.append(adj[i].columns.tolist())
cols = set([item for sublist in cols for item in sublist])

rows = []
for i in range(len(adj)):
    rows.append(adj[i].index.tolist())
rows = set([item for sublist in rows for item in sublist])

In [6]:
#pad out rows
for i in range(len(adj)):
    to_app = list(rows - set(adj[i].index.tolist()))
    for j in range(len(to_app)):
        name = to_app[j]
        series = pd.Series(list(np.zeros(len(adj[i].columns.tolist()))), index = adj[i].columns.tolist())
        series.name = name
        adj[i] = adj[i].append(series)

#rows looks good, will have to sort later
#now columns
#looks good!

for i in range(len(adj)):
    to_app = list(cols - set(adj[i].columns.tolist()))
    for j in range(len(to_app)):
        adj[i][to_app[j]] = 0

In [7]:
#all sorted, so hopefully now the node indexing will be consistent
for i in range(len(adj)):
    adj[i] = adj[i].sort_index()
    adj[i] = adj[i].sort_index(axis = 1)

In [8]:
adj[0].columns == adj[2].columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)